In [6]:
import * as React from "react";
import { useState, useEffect, useCallback } from "react";
import {
  Button,
  Text,
  Spinner,
  makeStyles,
  shorthands,
  tokens,
} from "@fluentui/react-components";
import * as FluentIcons from "@fluentui/react-icons";

const useStyles = makeStyles({
  container: {
    display: "flex",
    flexDirection: "column",
    width: "100%",
    height: "100vh",
    backgroundColor: tokens.colorNeutralBackground3,
    ...shorthands.padding("24px"),
    boxSizing: "border-box",
    overflow: "hidden",
  },
  actionBar: {
    display: "flex",
    flexDirection: "column",
    width: "100%",
    backgroundColor: tokens.colorNeutralBackground2,
    borderRadius: tokens.borderRadiusMedium,
    transition: "all 0.2s ease",
    boxShadow: tokens.shadow4,
    ":hover": {
      backgroundColor: tokens.colorNeutralBackground2Hover,
      transform: "translateY(-2px)",
      boxShadow: tokens.shadow8,
    },
  },
  actionContent: {
    display: "flex",
    flexDirection: "column",
    gap: "8px",
    padding: "16px",
  },
  actionTitle: {
    fontSize: tokens.fontSizeBase500,
    fontWeight: tokens.fontWeightSemibold,
    color: tokens.colorNeutralForeground1,
  },
  actionDescription: {
    fontSize: tokens.fontSizeBase300,
    color: tokens.colorNeutralForeground3,
  },
  iconWrapper: {
    display: "flex",
    alignItems: "center",
    justifyContent: "center",
    width: "32px",
    height: "32px",
    borderRadius: "50%",
    backgroundColor: "#106ebe",
    color: tokens.colorNeutralBackground1,
    marginRight: "12px",
  },
  input: {
    width: "100%",
    minHeight: "100px",
    resize: "vertical",
    backgroundColor: "transparent",
    border: `1px solid ${tokens.colorNeutralStroke1}`,
    borderRadius: tokens.borderRadiusMedium,
    padding: "8px",
    color: tokens.colorNeutralForeground1,
    ":focus": {
      outlineColor: tokens.colorBrandStroke1Focus,
    },
  },
  buttonContainer: {
    display: "flex",
    flexWrap: "wrap",
    gap: "8px",
    marginTop: "16px",
  },
  statusContainer: {
    display: "flex",
    justifyContent: "center",
    alignItems: "center",
    marginTop: "24px",
    minHeight: "60px",
  },
  statusMessage: {
    padding: "12px 24px",
    borderRadius: tokens.borderRadiusMedium,
    backgroundColor: tokens.colorNeutralBackground1,
    color: tokens.colorNeutralForeground1,
    boxShadow: tokens.shadow4,
    textAlign: "center",
    fontSize: tokens.fontSizeBase300,
    fontWeight: tokens.fontWeightSemibold,
    display: "flex",
    flexDirection: "column",
    alignItems: "center",
    justifyContent: "center",
    maxWidth: "80%",
  },
  successMessage: {
    backgroundColor: tokens.colorSuccessBackground,
    color: tokens.colorSuccessForeground1,
  },
  errorMessage: {
    backgroundColor: tokens.colorErrorBackground,
    color: tokens.colorErrorForeground1,
  },
  chatContainer: {
    marginTop: "24px",
    display: "flex",
    flexDirection: "column",
    gap: "16px",
    maxHeight: "calc(100vh - 400px)",
    overflowY: "auto",
  },
  chatMessage: {
    padding: "12px",
    borderRadius: tokens.borderRadiusMedium,
    maxWidth: "80%",
  },
  userMessage: {
    alignSelf: "flex-end",
    backgroundColor: tokens.colorBrandBackground,
    color: tokens.colorNeutralForegroundInverted,
  },
  assistantMessage: {
    alignSelf: "flex-start",
    backgroundColor: tokens.colorNeutralBackground1,
    color: tokens.colorNeutralForeground1,
  },
  chatInput: {
    marginTop: "16px",
    display: "flex",
    gap: "8px",
  },
  chatInputField: {
    flex: 1,
    padding: "8px",
    borderRadius: tokens.borderRadiusMedium,
    border: `1px solid ${tokens.colorNeutralStroke1}`,
  },
});

const EmailGenerator = ({ userId = "user1" }) => {
  const styles = useStyles();
  const [isLoading, setIsLoading] = useState(true);
  const [isProcessing, setIsProcessing] = useState(false);
  const [buttonConfig, setButtonConfig] = useState([]);
  const [statusMessage, setStatusMessage] = useState(null);
  const [error, setError] = useState(null);
  const [inputValue, setInputValue] = useState("");
  const [chatMessages, setChatMessages] = useState([]);
  const [chatInput, setChatInput] = useState("");
  const [selectedAction, setSelectedAction] = useState(null);

  const fetchButtonConfig = useCallback(async () => {
    setIsLoading(true);
    setError(null);
    try {
      console.log(`Fetching button configuration for user: ${userId}`);
      const response = await fetch(`http://localhost:8001/getButtonConfig/${userId}`);
      if (!response.ok) throw new Error(`HTTP error! status: ${response.status}`);
      const data = await response.json();
      console.log("Received button configuration:", data);
      setButtonConfig(data);
    } catch (e) {
      console.error("Error fetching button configuration:", e);
      setError(`Failed to load button configuration: ${e.message}`);
    } finally {
      setIsLoading(false);
    }
  }, [userId]);

  useEffect(() => {
    fetchButtonConfig();
  }, [fetchButtonConfig]);

  const getEmailContent = async () => {
    return new Promise((resolve, reject) => {
      Office.context.mailbox.item.body.getAsync(Office.CoercionType.Text, (result) => {
        if (result.status === Office.AsyncResultStatus.Succeeded) {
          resolve(result.value);
        } else {
          reject(new Error(result.error.message));
        }
      });
    });
  };

  const wrapInHtml = (content) => {
    // ... (keep the existing wrapInHtml function)
  };

  const handleButtonClick = (action) => {
    console.log(`Button clicked: ${action.label}, Requires input: ${action.requiresInput}`);
    if (action.requiresInput) {
      console.log("Setting selected action for chat");
      setSelectedAction(action);
      setChatMessages([{ role: "assistant", content: action.description }]);
    } else {
      console.log("Handling action directly");
      handleAction(action);
    }
  };

  const handleAction = async (action, userInput = null) => {
    setIsProcessing(true);
    setError(null);
    setStatusMessage(null);
    try {
      console.log(`Handling action: ${action.label}, User input: ${userInput}`);
      const content = await getEmailContent();
      const payload = {
        userId,
        emailContent: content,
        ...(userInput && { prompt: userInput }),
      };

      console.log(`Sending request to: ${action.apiEndpoint}`);
      const response = await fetch(action.apiEndpoint, {
        method: "POST",
        headers: { "Content-Type": "application/json" },
        body: JSON.stringify(payload),
      });

      if (!response.ok) throw new Error(`HTTP error! status: ${response.status}`);
      const responseData = await response.json();

      console.log("Received response:", responseData);
      const htmlContent = wrapInHtml(responseData.generatedContent || responseData.body);

      if (action.requiresInput) {
        setChatMessages(prevMessages => [
          ...prevMessages,
          { role: "assistant", content: responseData.generatedContent || responseData.body }
        ]);
      } else {
        Office.context.mailbox.item.displayReplyAllForm({
          htmlBody: htmlContent,
        });
        setStatusMessage({
          text: "Response generated successfully!",
        });
      }

      setInputValue("");
    } catch (e) {
      console.error(`Error in handleAction: ${e.message}`);
      setError(`Failed to ${action.label.toLowerCase()}. Please try again.`);
    } finally {
      setIsProcessing(false);
    }
  };

  const handleChatSubmit = (e) => {
    e.preventDefault();
    if (chatInput.trim() && selectedAction) {
      setChatMessages(prevMessages => [...prevMessages, { role: "user", content: chatInput }]);
      handleAction(selectedAction, chatInput);
      setChatInput("");
    }
  };

  if (isLoading) {
    return (
      <div className={styles.container}>
        <div className={styles.statusContainer}>
          <Spinner size="large" label="Loading button configuration..." />
        </div>
      </div>
    );
  }

  if (error) {
    return (
      <div className={styles.container}>
        <div className={styles.statusContainer}>
          <div className={`${styles.statusMessage} ${styles.errorMessage}`}>
            {error}
          </div>
        </div>
      </div>
    );
  }

  return (
    <div className={styles.container}>
      <div className={styles.actionBar}>
        <div className={styles.actionContent}>
          <div style={{ display: "flex", alignItems: "center" }}>
            <div className={styles.iconWrapper}>
              <FluentIcons.MailTemplate24Regular />
            </div>
            <Text className={styles.actionTitle}>Email Actions</Text>
          </div>
          <Text className={styles.actionDescription}>Select an action to perform on your email.</Text>
          <div className={styles.buttonContainer}>
            {buttonConfig.map((action, index) => (
              <Button
                key={index}
                appearance="primary"
                onClick={() => handleButtonClick(action)}
                disabled={isProcessing || selectedAction}
              >
                {action.label}
              </Button>
            ))}
          </div>
        </div>
      </div>
      {selectedAction && selectedAction.requiresInput && (
        <div className={styles.chatContainer}>
          {chatMessages.map((message, index) => (
            <div
              key={index}
              className={`${styles.chatMessage} ${
                message.role === "user" ? styles.userMessage : styles.assistantMessage
              }`}
            >
              {message.content}
            </div>
          ))}
          <form onSubmit={handleChatSubmit} className={styles.chatInput}>
            <input
              type="text"
              value={chatInput}
              onChange={(e) => setChatInput(e.target.value)}
              placeholder="Type your message..."
              className={styles.chatInputField}
            />
            <Button type="submit" disabled={isProcessing || !chatInput.trim()}>
              Send
            </Button>
          </form>
        </div>
      )}
      <div className={styles.statusContainer}>
        {isProcessing && (
          <div className={styles.statusMessage}>
            <Spinner size="tiny" />
            <span style={{ marginLeft: '8px' }}>Processing your request...</span>
          </div>
        )}
        {statusMessage && (
          <div className={`${styles.statusMessage} ${styles.successMessage}`}>
            <div>{statusMessage.text}</div>
          </div>
        )}
        {error && (
          <div className={`${styles.statusMessage} ${styles.errorMessage}`}>
            {error}
          </div>
        )}
      </div>
    </div>
  );
};

export default EmailGenerator;

ModuleNotFoundError: No module named 'streamlit'